# **Practical assignment for Topic 4**

**Link to your finished ipynb file in Google Colab:** *put the link here*

---

In your assigmnet, you will use the following **data**:

In [ ]:
X = np.array([-3.00, -2.54, -2.08, -1.62, -1.15, -0.69, -0.23,  0.23,  0.69,  1.15,  1.62,  2.08,  2.54,  3.00]).reshape(14, 1)
y = np.array([ 2.60,  2.81,  3.49,  4.08,  3.30,  3.43,  4.59,  4.65,  4.79,  6.45,  7.58,  9.78, 12.98, 15.49]).reshape(14, 1)

Your assignment consists of the following tasks:

1. Create a total of eight polynomials from degree 1 to 8. Compute and print training MSE and validation MSE values for each of them. Validation MSE should be computed using LOOCV (use the function that we already created). Draw a plot with two curves analogous to the plot shown in slide 9 from today's lecture. Add a legend so that it is clear which curve is which.
2. Repeat Task 1 with a modified LOOCV function so that it does not use anything from scikit-learn anymore - instead it should use only numpy. You should get the same result as in Task 1.
3. Explain the plot from Task 1. What exactly do we see in it and what conclusions can we make from it?
4. Draw another plot showing all datapoints of the given dataset together with the curves of all eight polynomials from degree 1 to 8. Each curve should be in a different color. Add a legend so that it is clear which curve is which.<br>
4.a. Do the task using scikit-learn.<br>
4.b. Do the task using numpy.

*While doing the tasks you are allowed to use only those function libraries and only those functions of numpy and scikit-learn libraries that we already have used in our notebooks of the course.*

**After the tasks are done, submit this file. Do not clear it's output - all print-outs and diagrams should be left in the file.**
